# PAP (Binary) - Models

## Load and preprocess data

In [2]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from transformers import DataCollatorWithPadding, get_scheduler
from datasets import load_dataset, Dataset, DatasetDict
from tqdm.auto import tqdm
import evaluate

# Loads PAP datasets
datasets_path = '../datasets/pap/train-dev-test-split/binary'
train_df = pd.read_csv(f'{datasets_path}/train.csv')
dev_df = pd.read_csv(f'{datasets_path}/dev.csv')
test_df = pd.read_csv(f'{datasets_path}/test.csv')

/Users/saponaro/Developer/uni/noble-owl/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loads and preprocess concreteness ratings:

In [3]:
# We got this dataset for concreteness of 40k words (https://pubmed.ncbi.nlm.nih.gov/24142837/) from https://web.stanford.edu/class/linguist278/data/
# Load concreteness ratings
concreteness_df = pd.read_csv('../datasets/concreteness/Concreteness_ratings_Brysbaert_et_al_BRM.csv')
concreteness_df.head(2)

# Map and normalize conreteness ratings
word_to_concreteness_score_map = dict()
for idx, row in concreteness_df.iterrows():
    row = row.to_dict()
    
    # Normalizing to a scale of 0 to 1
    word_to_concreteness_score_map[row['Word']] = row['Conc.M']/5.0 

Define helper functions to get concreteness scores:

In [4]:

def get_concreteness_score(word):
    """
    Get the concreteness score of a word based on the Concreteness Ratings dataset.
    """
    # If the word is not found in the dataset, return a default score of 0.5
    return round(word_to_concreteness_score_map.get(word, 0.5), 3)

def calculate_text_concreteness_sequence(text):
    """
    Calculate the concreteness score for a given text.
    """
    words = nltk.word_tokenize(text)
    concreteness_scores = [get_concreteness_score(word) for word in words]
    concreteness_scores = " ".join([str(i) for i in concreteness_scores])
    # Take the average concreteness score of all words in the text
    return concreteness_scores

Load PAP datasets:

In [5]:
# Add concreteness scores for the every sequence
train_df['concreteness_score_sequence'] = train_df.text.apply(calculate_text_concreteness_sequence)
dev_df['concreteness_score_sequence'] = dev_df.text.apply(calculate_text_concreteness_sequence)
test_df['concreteness_score_sequence'] = test_df.text.apply(calculate_text_concreteness_sequence)

# Load PAP datasets with Concreteness Scores  
raw_datasets = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(dev_df),
    'test': Dataset.from_pandas(test_df)
})

In [33]:
COLUMNS_TO_KEEP = ['label', 'input_ids', 'token_type_ids', 'attention_mask']

class ModellingExperiments:
    
    def __init__(self, model_name, dataset, batch_size, learning_rate):
        self.model_name = model_name
        self.dataset = dataset
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
        self.cols_to_keep = set(COLUMNS_TO_KEEP)
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.data_collator = DataCollatorWithPadding(self.tokenizer)
        self.model.to(self.device)
        
    def tokenize_sentence_with_concreteness_score(self, item):
        # We also tried using the concreteness score for the whole sentence as a feature input
        # To implement that, we changed the source code of transformers library and changed the classification head manually 
        # So that we can accomodate that extra feature, but this method of encoding concreteness score sequence was giving better score
        # Hence, we are using this only in the final experiments. You can check that experiment out in the following notebook:
        # modelling/pap/experiments/FinalModellingWithConcretenessScore[BERT] - PAP
        return self.tokenizer(item['text'], item['concreteness_score_sequence'], truncation=True)
        
    def tokenize_sentence(self, item):
        # Normal tokenization
        return self.tokenizer(item['text'], truncation=True)
        
    def add_strategy_to_tokenizer_function_map(self):
        # Mapping between strategy and the tokenization functions defined above
        # Strategy refers to whether we are using normal tokenization or whether we want to do paired tokenization of 
        # both input sentence and the sequence of concreteness score for that sentence
        self.strategy_to_tokenizer_function_map = dict()
        self.strategy_to_tokenizer_function_map['normal_finetuning'] = self.tokenize_sentence_with_concreteness_score
        self.strategy_to_tokenizer_function_map['concreteness_score_addition'] = self.tokenize_sentence
        
    def prepare_dataset(self, strategy):
        # Here, we wull tokenize the dataset based on the strategy we are planning to use
        self.strategy = strategy
        self.add_strategy_to_tokenizer_function_map()
        self.tokenized_dataset = self.dataset.map(self.strategy_to_tokenizer_function_map[self.strategy], batched=True)
        current_cols = set(list(self.tokenized_dataset['train'].features.keys()))
        self.tokenized_dataset = self.tokenized_dataset.remove_columns(list(current_cols - self.cols_to_keep))
        self.tokenized_dataset = self.tokenized_dataset.rename_column("label", "labels")
        self.tokenized_dataset = self.tokenized_dataset.with_format("torch")

    def prepare_dataloaders(self):
        self.train_dataloader = DataLoader(self.tokenized_dataset['train'], batch_size=self.batch_size, shuffle=True, collate_fn=self.data_collator)
        self.validation_dataloader = DataLoader(self.tokenized_dataset['validation'], batch_size=self.batch_size, collate_fn=self.data_collator)
        self.test_dataloader = DataLoader(self.tokenized_dataset['test'], batch_size=self.batch_size, collate_fn=self.data_collator)

    def setup_optimizer(self, num_epochs):
        # Setting up optimizer and learning rate scheduler
        self.num_epochs = num_epochs
        self.optimizer = AdamW(self.model.parameters(), lr=self.learning_rate)
        self.num_training_steps = self.num_epochs*len(self.train_dataloader)
        self.learning_rate_scheduler = get_scheduler("linear", optimizer=self.optimizer, num_warmup_steps=0, num_training_steps=self.num_training_steps)
    
    def train_model(self):
        # Training the Model
        self.model.train()
        progress_bar = tqdm(range(self.num_training_steps))
        for epoch in range(self.num_epochs):
            for batch in self.train_dataloader:
                batch = {k:v.to(self.device) for k, v in batch.items()}
                outputs = self.model(**batch)
                loss = outputs.loss
                # calculating gradients
                loss.backward()
                # optimizing weights
                self.optimizer.step()
                # updating learning rate
                self.learning_rate_scheduler.step()
                # flushing gradients
                self.optimizer.zero_grad()
                # updating progress bar
                progress_bar.update(1)

    def initialize_metrics(self):
        self.metrics = {
            'accuracy': evaluate.load('accuracy'),
            'precision': evaluate.load('precision'),
            'recall': evaluate.load('recall'),
            'f1': evaluate.load('f1'),
            'roc-auc': evaluate.load("roc_auc"),
        }
                
    def eval_model(self, dataloader):
        # Run on GPU available
        #device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        
        # Evaluating the model on different dataloaders
        self.initialize_metrics()
        
        self.model.eval()
        
        for batch in dataloader:
            # Move batch data to the specified device (GPU or CPU)
            batch = {k: v.to(self.device) for k, v in batch.items()}
    
            # Forward pass
            with torch.no_grad():
                outputs = self.model(**batch)
            
            # Extract logits and predictions
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
        
            # Apply softmax to convert logits to probabilities
            probabilities = torch.nn.functional.softmax(logits, dim=-1)
            
            # Extract probabilities for the positive class
            positive_probabilities = probabilities[:, 1].to(self.device).numpy()
        
            # Update metrics for accuracy, precision, recall, F1 and ROC-AUC
            self.metrics['accuracy'].add_batch(predictions=predictions, references=batch['labels'])
            self.metrics['precision'].add_batch(predictions=predictions, references=batch['labels'])
            self.metrics['recall'].add_batch(predictions=predictions, references=batch['labels'])
            self.metrics['f1'].add_batch(predictions=predictions, references=batch['labels'])
            self.metrics['roc-auc'].add_batch(prediction_scores=positive_probabilities, references=batch['labels'])
        
        # Compute metrics for accuracy, precision, recall, F1 and ROC-AUC
        self.eval_dict = {}
        self.eval_dict.update(self.metrics['accuracy'].compute())
        self.eval_dict.update(self.metrics['precision'].compute(average="macro"))
        self.eval_dict.update(self.metrics['recall'].compute(average="macro"))
        self.eval_dict.update(self.metrics['f1'].compute(average="macro"))
        self.eval_dict.update(self.metrics['roc-auc'].compute(average="macro"))     

## Run Experiments

Defining parameters on which we will run the experiments

In [26]:
# Initialize model parameters
model_name_list = ["facebook/bart-base", "microsoft/deberta-base"]
num_epochs_list = [1, 2, 3, 4, 5]
strategies_list = ["normal_finetuning", "concreteness_score_addition"]

Defining static arguments

In [27]:
kw_args = {
    'dataset': raw_datasets,
    'batch_size': 32,
    'learning_rate': 3e-5,
}

### Experiments loop

In [34]:
result_list = list()
for model_name in model_name_list:
    
    # Setting Model Name
    kw_args["model_name"] = model_name
    
    # Initializing ModellingExperiments Object
    modelling_obj = ModellingExperiments(**kw_args)
    
    for strategy in strategies_list:
        
        # Preparing dataset for a specific strategy
        modelling_obj.prepare_dataset(strategy=strategy)
        
        # Preparing data loaders
        modelling_obj.prepare_dataloaders()
            
        # Training loop
        for num_epochs in num_epochs_list:
            # Initializing dictionary for storing results
            result_dict = dict()
            result_dict["model_name"] = model_name
            result_dict["strategy"] = strategy
            #result_dict["train_dataset_type"] = train_dataset_type
            result_dict["num_epochs"] = num_epochs
            print("Model Training with the following Configurations: {}".format(result_dict))
            
            # For a specic num_epochs variable, we are setting up the optimizers
            modelling_obj.setup_optimizer(num_epochs=num_epochs)
            
            # Now, we are training the model
            modelling_obj.train_model()
            
            # Now, we will evaluate the model on validation dataset
            modelling_obj.eval_model(modelling_obj.validation_dataloader)
            
            # Storing results on validation set
            for k, v in modelling_obj.eval_dict.items():
                result_dict["validation_{}".format(k)] = v
            print("Validation Set Results: {}".format(modelling_obj.eval_dict))
            
            # Now, we will evaluate the model on test dataset
            modelling_obj.eval_model(modelling_obj.test_dataloader)
            
            # Storing results on test set
            for k, v in modelling_obj.eval_dict.items():
                result_dict["test_{}".format(k)] = v
            
            print("Test Set Results: {}".format(modelling_obj.eval_dict))
            
            # Storing all the results in the results_list
            result_list.append(result_dict)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 216/216 [00:00<00:00, 42138.12 examples/s]


Model Training with the following Configurations: {'model_name': 'facebook/bart-base', 'strategy': 'normal_finetuning', 'num_epochs': 1}


100%|██████████| 54/54 [01:01<00:00,  1.14s/it]
/Users/saponaro/Developer/uni/noble-owl/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Set Results: {'accuracy': 0.7129629629629629, 'precision': 0.35648148148148145, 'recall': 0.5, 'f1': 0.41621621621621624, 'roc_auc': 0.5528906577293674}


/Users/saponaro/Developer/uni/noble-owl/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Validation Set Results: {'accuracy': 0.7129629629629629, 'precision': 0.6369047619047619, 'recall': 0.6156263091746963, 'f1': 0.6219085262563524, 'roc_auc': 0.7132383745286971}
Test Set Results: {'accuracy': 0.7222222222222222, 'precision': 0.6491048593350384, 'recall': 0.6221198156682027, 'f1': 0.6296296296296295, 'roc_auc': 0.7328236279849184}
Model Training with the following Configurations: {'model_name': 'microsoft/deberta-base', 'strategy': 'concreteness_score_addition', 'num_epochs': 4}


100%|██████████| 216/216 [03:10<00:00,  1.13it/s]


Validation Set Results: {'accuracy': 0.7083333333333334, 'precision': 0.6292962356792144, 'recall': 0.607561793045664, 'f1': 0.6134859544976852, 'roc_auc': 0.7270632593213238}
Test Set Results: {'accuracy': 0.7222222222222222, 'precision': 0.6502976190476191, 'recall': 0.6269375785504818, 'f1': 0.6341050254093732, 'roc_auc': 0.7204650188521157}
Model Training with the following Configurations: {'model_name': 'microsoft/deberta-base', 'strategy': 'concreteness_score_addition', 'num_epochs': 5}


100%|██████████| 270/270 [03:57<00:00,  1.13it/s]


Validation Set Results: {'accuracy': 0.7175925925925926, 'precision': 0.6443235977025541, 'recall': 0.6236908253037285, 'f1': 0.6302096489012377, 'roc_auc': 0.7269585253456221}
Test Set Results: {'accuracy': 0.7314814814814815, 'precision': 0.6636904761904762, 'recall': 0.6382488479262672, 'f1': 0.6463015245623942, 'roc_auc': 0.6936531210724759}


In [35]:
results_df = pd.DataFrame(result_list)
results_df.head()

,model_name,strategy,num_epochs,validation_accuracy,validation_precision,validation_recall,validation_f1,validation_roc_auc,test_accuracy,test_precision,test_recall,test_f1,test_roc_auc
0,facebook/bart-base,normal_finetuning,1,0.712963,0.356481,0.500000,0.416216,0.552891,0.712963,0.356481,0.500000,0.416216,0.543884
1,facebook/bart-base,normal_finetuning,2,0.754630,0.716931,0.615941,0.624988,0.792417,0.685185,0.555466,0.528697,0.514799,0.725073
2,facebook/bart-base,normal_finetuning,3,0.740741,0.683891,0.601382,0.607018,0.768538,0.731481,0.662366,0.594889,0.599488,0.740050
3,facebook/bart-base,normal_finetuning,4,0.671296,0.555111,0.538228,0.534808,0.702870,0.754630,0.701872,0.640029,0.652744,0.742250
4,facebook/bart-base,normal_finetuning,5,0.680556,0.553292,0.530268,0.519706,0.681609,0.708333,0.622121,0.588291,0.592953,0.713553


In [36]:
results_df.to_csv('../../results/FinalResultsPAP.csv', index=False)